In [ ]:
#imported
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
import json
import numpy
import os
import shutil
import tensorflow as tf

In [ ]:
def import_data(jfile='/work/Legion - The AI powered ChatBot/intents.json'):
    '''
    This will load intents file
    intents={tag,patterns,responses,context_set}
    '''
    with open(jfile) as file1:
        ds=json.load(file1)
    return ds

In [ ]:

def tokenize_words(ds):
    words=[]
    intents_x=[]
    intent_tags=[]
    intents_y=[] 
    for intents in ds:
        #print(intents)
        intent_tags.append(intents['tag'])
        for t in intents['patterns']+intents['responses']: #to consider Q&A both
            tokens=nltk.regexp_tokenize(t,"[\w']+")
            words.extend(tokens)
            intents_x.append(tokens)
            intents_y.append(intents["tag"])
       
    
        
    intent_tags=sorted(intent_tags)
    return (words,intents_x,intents_y,intent_tags)
    

In [ ]:
data=import_data()
print(data)

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there', 'namaste', 'Hello', 'Good day', 'Good Morning', 'Whats Up', 'Good Day', 'Good Afternoon', 'Good Evening'], 'responses': ['Hello!', 'Good to See you Again!', 'Hi there,how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'Goodbye', 'Bye', 'I am Leaving', 'Have a Good Day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'GoodBye!', 'Nice to talk to you,Bye'], 'context_set': ''}, {'tag': 'age', 'patterns': ['how old', 'How old are you?', 'What is your age', 'How old are you', 'age?'], 'responses': ['I am ageless. I am eternal.', 'I am eternal', 'I am forever. '], 'context_set': ''}, {'tag': 'name', 'patterns': ['what is your name', 'What should i call you', 'whats your name', 'name,please?'], 'responses': ['You can call me Legion', 'I am Legion', "I'm Legion -  The AI powered ChatBot "], 'context_set': ''}, {'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone t

In [ ]:
words,xdata,ydata,tag=tokenize_words(data)
print("words",words)
print("x:",xdata)
print("y:",ydata)
print("t:",tag)


words ['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', 'namaste', 'Hello', 'Good', 'day', 'Good', 'Morning', 'Whats', 'Up', 'Good', 'Day', 'Good', 'Afternoon', 'Good', 'Evening', 'Hello', 'Good', 'to', 'See', 'you', 'Again', 'Hi', 'there', 'how', 'can', 'I', 'help', 'cya', 'Goodbye', 'Bye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'Day', 'Sad', 'to', 'see', 'you', 'go', 'Talk', 'to', 'you', 'later', 'GoodBye', 'Nice', 'to', 'talk', 'to', 'you', 'Bye', 'how', 'old', 'How', 'old', 'are', 'you', 'What', 'is', 'your', 'age', 'How', 'old', 'are', 'you', 'age', 'I', 'am', 'ageless', 'I', 'am', 'eternal', 'I', 'am', 'eternal', 'I', 'am', 'forever', 'what', 'is', 'your', 'name', 'What', 'should', 'i', 'call', 'you', 'whats', 'your', 'name', 'name', 'please', 'You', 'can', 'call', 'me', 'Legion', 'I', 'am', 'Legion', "I'm", 'Legion', 'The', 'AI', 'powered', 'ChatBot', 'Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Hello', 'thanks', 'fo

In [ ]:

def stemming(words):
    stemmer=nltk.stem.LancasterStemmer()
    stem_words=[]
    for w in words:
        stem_words.append(stemmer.stem(w.lower()))
    root_words=sorted(set(stem_words))
    return root_words




In [ ]:
root=stemming(words)
print (root)

["'armageddon'", '00am', '10', '15', '1st', '2', '20', '23a12', '23a31', '25', '2nd', '30pm', '32712', '345a23', '360', '36c', '3rd', '40', '431b67', '5', '561a24', '562b78', '7', '9', 'a', 'abc', 'abl', 'about', 'abx', 'ac', 'access', 'accid', 'accord', 'accuweath', 'adv', 'advers', 'afternoon', 'ag', 'again', 'ageless', 'ai', 'al', 'along', 'am', 'an', 'analys', 'and', 'any', 'anyon', 'app', 'apply', 'appoint', 'appoit', 'ar', 'arrest', 'as', 'ask', 'at', 'avail', 'aw', 'away', 'awesom', 'b', 'back', 'bangl', 'bar', 'bas', 'be', 'been', 'benefit', 'berlin', 'bhat', 'block', 'bluetoo', 'bor', 'bruno', 'build', 'by', 'bye', 'cabin', 'cafeter', 'cal', 'can', "can't", 'canteen', 'capac', 'car', 'chain', 'challeng', 'champ', 'chang', 'chat', 'chatbot', 'check', 'chin', 'clar', 'clear', 'clock', 'com', 'commit', 'company', 'compens', 'complaint', 'compon', 'compr', 'comput', 'config', 'conflict', 'consum', 'contact', 'control', 'cost', 'could', 'cre', 'crit', 'custom', 'cya', 'd', 'dat', '

In [ ]:
def train_data(root,xdata,ydata,tag):
    stemmer=nltk.stem.LancasterStemmer()
    train=[]
    output=[]
    for i in range(len(xdata)):
        #for x
        words_bag=[]
        temp_words=[]
        for w in xdata[i]:
            temp_words.append(stemmer.stem(w.lower()))
        for w in root:
            found=1 if w in temp_words else 0
            words_bag.append(found)
        train.append(words_bag)
        # for y
        row=[0] * len(tag)
        row[tag.index(ydata[i])]=1
        output.append(row)
    train_x=numpy.array(train)
    train_y=numpy.array(output)
    return (train_x,train_y)

In [ ]:
t,o=train_data(root,xdata,ydata,tag)
print("t:",t.shape)
print("o:",o.shape)

t: (244, 467)
o: (244, 50)


In [ ]:
#build model
def build_model(train_x,train_y):
    epochs=1000
    batch_size=16
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(batch_size,input_shape=(t.shape[1],)))
    model.add(tf.keras.layers.Dense(batch_size)) #hidden layer
    model.add(tf.keras.layers.Dense(batch_size)) #hidden layer
    model.add(tf.keras.layers.Dense(len(train_y[0]),activation="softmax")) #output layer
    model.compile(optimizer= "adam",loss="categorical_crossentropy",metrics=["accuracy"])
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=0)
    #model evaluation
    model.summary()
    test_loss,test_acc=model.evaluate(train_x,train_y,verbose=0)
    print("\nLoss:{} Acc:{}".format(test_loss,test_acc))
    return model
    






In [ ]:
build_model(t,o)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                7488      
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 50)                850       
                                                                 
Total params: 8,882
Trainable params: 8,882
Non-trainable params: 0
_________________________________________________________________

Loss:0.013565974310040474 Acc:0.9918032884597778


In [ ]:
def prepare_model(ds,model_file="/work/Legion - The AI powered ChatBot/model.chatbot"):
    
    words,xdata,ydata,tag=tokenize_words(data)
    
    root=stemming(words)
    if os.path.isdir(model_file):
        
        shutil.rmtree('model.chatbot')
        t,o=train_data(root,xdata,ydata,tag)
        model=build_model(t,o)
        model.save(model_file)
        print("Loading model Successfully")
    else:
        
        t,o=train_data(root,xdata,ydata,tag)
        model=build_model(t,o)
        model.save(model_file)
    return (model,root,tag)


In [ ]:
model,root,tag=prepare_model(data)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                7488      
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 50)                850       
                                                                 
Total params: 8,882
Trainable params: 8,882
Non-trainable params: 0
_________________________________________________________________

Loss:0.013556286692619324 Acc:0.9918032884597778
INFO:tensorflow:Assets written to: /work/Legion - The AI powered ChatBot/model.chatbot/assets
Loading model Successfully


In [ ]:
def chat(model,root,tag,data):
    print("Lets Talk with me(Type exit to stop)")
    while(True):
        ques=input("You:")
        if ques.lower()=="exit":break;
        

        tokens=nltk.regexp_tokenize(ques,"[\w']+")
        
        
        stemmer=nltk.stem.LancasterStemmer()
        stem_tokens=[]
        for token in tokens:
            stem_tokens.append(stemmer.stem(token.lower()))
        
        bag_of_words=[]
        for w in root:
            found=1 if w in stem_tokens else 0
         
            bag_of_words.append(found)
        
        print(len(bag_of_words))
        prediction=model.predict(numpy.array([bag_of_words]))
        max_index=numpy.argmax(prediction)
        max_probabilities=prediction[0][max_index]
        print(ques)
        if max_probabilities < 0.8:
            print ("Sorry, I don't know this.(confidence:{})".format(max_probabilities))
            continue
        else:
            found_tag=tag[max_index]   

        for intents in data:
            if intents['tag']==found_tag:
                print("{} (confidence: {})".format(numpy.random.choice(intents['responses']),max_probabilities))
                    

                

In [ ]:
chat(model,root,tag,data)

Lets Talk with me(Type exit to stop)
467
1/1 [==============================] - 0s 224ms/step
Tell me jo
Sorry, I don't know this.(confidence:0.7728855609893799)
467
1/1 [==============================] - 0s 77ms/step
Tell me a joke
Never criticize someone until you've walked a mile in their shoes. That way, when you criticize them, they won't be able to hear you from that far away. Plus, you'll have their shoes. (confidence: 0.999995231628418)
467
1/1 [==============================] - 0s 153ms/step
Tell me a joke
I ate a clock yesterday, it was very time-consuming (confidence: 0.999995231628418)
467
1/1 [==============================] - 0s 60ms/step
Tel
Can't Understand Your Question (confidence: 0.9971407651901245)
467
1/1 [==============================] - 0s 52ms/step
Tell me a jok
o what if I don't know what 'Armageddon' means? It's not the end of the world. (confidence: 0.999995231628418)
467
1/1 [==============================] - 0s 51ms/step
Tell me a jok
What did the traffic

In [ ]:

 
rootdir = '/work/Legion - The AI powered ChatBot'
for it in os.scandir(rootdir):
    if it.is_dir():
        print(it.path)

/work/Legion - The AI powered ChatBot/model.chatbot


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ac3d1c9e-c820-4000-97f9-b25922505111' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>